In [169]:
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock)
import idaes.core.util.scaling as iscale
from idaes.models_extra.power_generation.properties.natural_gas_PR import (
    get_prop,
    EosType,
)
from idaes.core.util.model_statistics import (report_statistics,
                                              unfixed_variables_set,
                                              degrees_of_freedom)
from idaes_ui.fv import visualize 

In [ ]:
import importlib
import SOFC_files.SPEZZATINO_SOFC.SOFC_unit
importlib.reload(SOFC_files.SPEZZATINO_SOFC.SOFC_unit)

from SOFC_files.SPEZZATINO_SOFC.SOFC_unit import SofcUnit

def sofc_example_flowsheet(eos=EosType.PR):
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    # --- property packages ---
    fuel_comps      = {"H2": 0.99, "H2O": 0.01}
    air_comps       = {"O2": 0.21, "N2": 0.79}

    m.fs.cathode_side_prop_package = GenericParameterBlock(
        **get_prop(air_comps, {"Vap"}, eos=eos),
        doc="Air property parameters",
    )
    m.fs.anode_side_prop_package = GenericParameterBlock(
        **get_prop(fuel_comps, {"Vap"}, eos=eos),
        doc="Flue gas property parameters",
    )
    m.fs.sofc = SofcUnit(
        cathode_side_prop_package=m.fs.cathode_side_prop_package,
        anode_side_prop_package=m.fs.anode_side_prop_package,
        reaction_eos=eos
    )


    # design / operating points
    m.fs.sofc.fuel_util.fix(0.7)
    m.fs.sofc.number_of_cells.fix(80)
    m.fs.sofc.cell_area.fix(320)

    # 1) H₂ translator inlet
    m.fs.sofc.anode_side_inlet.flow_mol.fix(0.075397)
    m.fs.sofc.anode_side_inlet.temperature.fix(942.05)
    m.fs.sofc.anode_side_inlet.pressure.fix(101_325)
    m.fs.sofc.anode_side_inlet.mole_frac_comp[0, "H2"].fix(0.99)
    m.fs.sofc.anode_side_inlet.mole_frac_comp[0, "H2O"].fix(0.01)

    # 2) Air separator inlet
    m.fs.sofc.cathode_side_inlet.flow_mol.fix(2)
    m.fs.sofc.cathode_side_inlet.temperature.fix(948.45)
    m.fs.sofc.cathode_side_inlet.pressure.fix(101_325)
    m.fs.sofc.cathode_side_inlet.mole_frac_comp[0, "O2"].fix(0.21)
    m.fs.sofc.cathode_side_inlet.mole_frac_comp[0, "N2"].fix(0.79)
    
    return m

# create a flowsheet with the sofc model
model = sofc_example_flowsheet()


In [171]:
print(degrees_of_freedom(model))

0


In [172]:
from pyomo.util.infeasible import log_infeasible_constraints

#iscale.calculate_scaling_factors(model)
model.fs.sofc.initialize(optarg={"max_iter":30})
print("\n----- MODEL STATISTICS -----")
print(report_statistics(model))

solver = pyo.SolverFactory("ipopt")
solver.options = {"tol": 1e-6, "max_iter": 1_000, "print_level": 0}
results = solver.solve(model, tee=True)
log_infeasible_constraints(model, log_expression=True)


2025-06-09 17:56:08 [INFO] idaes.init.fs.sofc.translator_h2.properties_in: Starting initialization
2025-06-09 17:56:08 [INFO] idaes.init.fs.sofc.translator_h2.properties_in: Property initialization: optimal - Optimal Solution Found.
2025-06-09 17:56:08 [INFO] idaes.init.fs.sofc.translator_h2.properties_out: Starting initialization
2025-06-09 17:56:09 [INFO] idaes.init.fs.sofc.translator_h2.properties_out: Property initialization: optimal - Optimal Solution Found.
2025-06-09 17:56:09 [INFO] idaes.init.fs.sofc.translator_h2.properties_out: Property package initialization: optimal - Optimal Solution Found.
2025-06-09 17:56:09 [INFO] idaes.init.fs.sofc.translator_h2: Initialization Complete optimal - Optimal Solution Found.
2025-06-09 17:56:09 [INFO] idaes.init.fs.sofc.separator.mixed_state: Starting initialization
2025-06-09 17:56:09 [INFO] idaes.init.fs.sofc.separator.mixed_state: Property initialization: optimal - Optimal Solution Found.
2025-06-09 17:56:09 [INFO] idaes.init.fs.sofc.sep

In [173]:
from pyomo.environ import value

visualize(model.fs.sofc, "My Process Flowsheet")


print(value(model.fs.sofc.i[:]))              # current density [A m⁻²]
print(value(model.fs.sofc.cell_voltage[:]))   # cell voltage [V]
print(value(model.fs.sofc.sofc_power_dc[:]))  # DC power [W]


2025-06-09 17:56:11 [INFO] idaes.idaes_ui.fv.fsvis: Using HTTP server on localhost, port 63375
2025-06-09 17:56:11 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'My Process Flowsheet.json'
2025-06-09 17:56:11 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'My Process Flowsheet.json' in current directory (c:\Users\gcasagrande\Desktop\Repo_TesiSara\idaes_sofc)
2025-06-09 17:56:11 [WARNING] idaes.idaes_ui.fv.fsvis: Flowsheet name changed: old='My Process Flowsheet' new='My-Process-Flowsheet'
Flowsheet name changed to 'My-Process-Flowsheet'
2025-06-09 17:56:11 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:63375/app?id=My-Process-Flowsheet
[0.00020450845816309582]
[1.1832693013835263]
[6.194907658692313]


In [174]:
from idaes.core.util.model_statistics import (
    degrees_of_freedom, 
    report_statistics,
    activated_constraints_set,
    activated_equalities_set,
    unfixed_variables_set,
    fixed_variables_set,
    variables_set,
)


def report_model_dof(model, stream_label, stream):
    separator = '=' * 60
    subsection = '-' * 60

    def format_var(i, var):
        try:
            val = value(var)
            val_str = f"{val: .6e}" if abs(val) < 1e5 else f"{val: .3f}"
        except:
            val_str = "[NaN]"
        return f"{i:>3} - {val_str:>14}   {var}"

    print(f"\n{separator}")
    print(f"{stream_label:^60}")
    print(f"{separator}")

    print("\nSTREAM PROPERTIES:")
    print(f"{'Flow [mol/s]':<20}: {value(stream.flow_mol[0]):.6e}")
    print(f"{'Temperature [K]':<20}: {value(stream.temperature[0]):.2f}")
    print(f"{'Pressure [Pa]':<20}: {value(stream.pressure[0]):.2f}")
    try:
        print(f"{'Heat [W]':<20}: {value(model.control_volume.heat[0]):.2f}")
    except:
        pass

    try:
        print(f"{'Heat [W]':<20}: {value(model.control_volume.heat_of_reaction[0]):.2f}")
    except:
        pass
    
    
    # print("\nMOLE FRACTIONS:")
    # for t, c in stream.mole_frac_comp.keys():
    #     print(f"  x_{c:<10}: {value(stream.mole_frac_comp[t, c]):.6e}")

    # print(f"\nDOF after {stream_label}: {degrees_of_freedom(model)}")

    # print(f"\n{subsection}")
    # print("MODEL STATISTICS REPORT:")
    # print(subsection)
    # report_statistics(model)

    # print(f"\n{subsection}")
    # print("VARIABLES:")
    # print(subsection)
    # for i, v in enumerate(variables_set(model)):
    #     print(format_var(i + 1, v))

    # print(f"\n{subsection}")
    # print("ACTIVATED CONSTRAINTS:")
    # print(subsection)
    # for i, c in enumerate(activated_constraints_set(model)):
    #     print(f"{i + 1:>3} - {c}")

    # print(f"\n{subsection}")
    # print("ACTIVATED EQUALITIES:")
    # print(subsection)
    # for i, eq in enumerate(activated_equalities_set(model)):
    #     print(f"{i + 1:>3} - {eq}")

    # print(f"\n{subsection}")
    # print("UNFIXED VARIABLES:")
    # print(subsection)
    # for i, ufv in enumerate(unfixed_variables_set(model)):
    #     print(format_var(i + 1, ufv))

    # print(f"\n{subsection}")
    # print("FIXED VARIABLES:")
    # print(subsection)
    # for i, fv in enumerate(fixed_variables_set(model)):
    #     print(format_var(i + 1, fv))

In [175]:
# Esempio di richiamo strutturato per tutte le unità e i flussi associati

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_h2, "Translator H2 - Inlet", model.fs.sofc.translator_h2.inlet)
report_model_dof(model.fs.sofc.translator_h2, "Translator H2 - Outlet", model.fs.sofc.translator_h2.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator Air (O2 Rich e O2 Poor)
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.separator, "Separator Air - Inlet", model.fs.sofc.separator.inlet)
report_model_dof(model.fs.sofc.separator, "Separator Air - O2 Rich", model.fs.sofc.separator.o2_rich_strm)
report_model_dof(model.fs.sofc.separator, "Separator Air - O2 Poor", model.fs.sofc.separator.o2_poor_strm)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator O2-rich da separator a reaction_props
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_o2, "Translator O2 - Inlet", model.fs.sofc.translator_o2.inlet)
report_model_dof(model.fs.sofc.translator_o2, "Translator O2 - Outlet", model.fs.sofc.translator_o2.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per H2 e O2 inlet
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.mixer, "Mixer - O2 Inlet", model.fs.sofc.mixer.o2_rich_strm)
report_model_dof(model.fs.sofc.mixer, "Mixer - H2 Inlet", model.fs.sofc.mixer.fuel_strm)
report_model_dof(model.fs.sofc.mixer, "Mixer - Mixed Outlet", model.fs.sofc.mixer.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Stoichiometric Reactor
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.reactor, "Reactor - Inlet", model.fs.sofc.reactor.inlet)
report_model_dof(model.fs.sofc.reactor, "Reactor - Outlet", model.fs.sofc.reactor.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator water e O2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.reactor_separator, "Separator Reactor - O2 Stream", model.fs.sofc.reactor_separator.o2_strm)
report_model_dof(model.fs.sofc.reactor_separator, "Separator Reactor - Water Stream", model.fs.sofc.reactor_separator.water_strm)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2O e H2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_h2_out, "Translator H2 Out - Inlet", model.fs.sofc.translator_h2_out.inlet)
report_model_dof(model.fs.sofc.translator_h2_out, "Translator H2 Out - Outlet", model.fs.sofc.translator_h2_out.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator O2-rich → air
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_o2_out, "Translator O2 Out - Inlet", model.fs.sofc.translator_o2_out.inlet)
report_model_dof(model.fs.sofc.translator_o2_out, "Translator O2 Out - Outlet", model.fs.sofc.translator_o2_out.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Heater condizionamento finale
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.heater, "Heater - Inlet", model.fs.sofc.heater.inlet)
report_model_dof(model.fs.sofc.heater, "Heater - Outlet", model.fs.sofc.heater.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per O2 riciclato e N2 in uscita
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - O2 Inlet", model.fs.sofc.mixer_out.o2_strm_final)
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - N2 Inlet", model.fs.sofc.mixer_out.o2_poor_strm)
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - Mixed Outlet", model.fs.sofc.mixer_out.outlet)



                   Translator H2 - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 7.539700e-02
Temperature [K]     : 942.05
Pressure [Pa]       : 202650.00

                   Translator H2 - Outlet                   

STREAM PROPERTIES:
Flow [mol/s]        : 7.539700e-02
Temperature [K]     : 942.05
Pressure [Pa]       : 202650.00

                   Separator Air - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 2.000000e+00
Temperature [K]     : 948.45
Pressure [Pa]       : 202650.00

                  Separator Air - O2 Rich                   

STREAM PROPERTIES:
Flow [mol/s]        : 4.200000e-01
Temperature [K]     : 948.45
Pressure [Pa]       : 202650.00

                  Separator Air - O2 Poor                   

STREAM PROPERTIES:
Flow [mol/s]        : 1.580000e+00
Temperature [K]     : 948.45
Pressure [Pa]       : 202650.00

                   Translator O2 - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 4.2000